In [ ]:
# install some packages

# for easy downloading from google drive
!pip install gdown 

# for loading parquet
!pip install pyarrow
!pip install fastparquet

# for models
!pip install pytorch_lightning
!pip install pytorch-forecasting

In [1]:
import os
import sys
import pickle


import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as ttf

import os
import os.path as osp

from tqdm import tqdm
from sklearn.metrics import roc_auc_score


# Download Data & Create Test DataFrame

In [2]:
if False: # download unprocessed data in parquet form
    !kaggle datasets download -d robikscube/ubiquant-parquet -p /home/ubuntu/data/ubiquant
    !unzip -q /home/ubuntu/data/ubiquant/ubiquant-parquet.zip -d /home/ubuntu/data/ubiquant

In [3]:
if False:
    !gdown 1yhCV0v-GgYAfhKnJptvqSZdivvnAa27M -O /home/ubuntu/data/dyj_byid.zip
    !unzip -q /home/ubuntu/data/dyj_byid.zip -d /home/ubuntu/data

In [4]:
df = pd.read_parquet('/home/ubuntu/data/ubiquant/train_low_mem.parquet')

In [30]:
#!gdown 1QBvxnH4Jb_wZ5lMf1ATSssEwo3UTQmQn -O /content/ubiquant/test_500_ids.pkl

with open('/home/ubuntu/efs/project/src/data/test_500_ids.pkl', 'rb') as f:
    test_500_ids = pickle.load(f)
with open('/home/ubuntu/efs/project/src/data/test_1000_ids.pkl', 'rb') as f:
    test_1000_ids = pickle.load(f)
with open('/home/ubuntu/efs/project/src/data/50_test_ids.pkl', 'rb') as f:
    test_50_ids = pickle.load(f)

In [31]:
test_df = df[(df['investment_id'].isin(test_50_ids)) & (df['time_id'] >= 1100)]

In [32]:
test_df.investment_id = test_df.investment_id.astype(str)
test_df.time_id = test_df.time_id.astype(int)
test_df.dtypes

row_id            object
time_id            int64
investment_id     object
target           float32
f_0              float32
                  ...   
f_295            float32
f_296            float32
f_297            float32
f_298            float32
f_299            float32
Length: 304, dtype: object

# Load Model

In [33]:
import warnings
warnings.filterwarnings('ignore')

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning.loggers import WandbLogger


In [ ]:
# Download Trained Model, DataSet and 
if False:
    ! gdown --folder 1cVr_V6KOoSyL_Ry-PUOphRW74En1Rq04 -O /home/ubuntu/data/trained_model/

In [35]:
train_dataset = TimeSeriesDataSet.load("/home/ubuntu/data/trained_model/TFT_429_tune_3/tft_train_100_samples.pf")
print("Load train dataset completed")

Load train dataset completed


In [34]:
tft_model = TemporalFusionTransformer.load_from_checkpoint("/home/ubuntu/data/trained_model/TFT_429_tune_3/Linux_429_3_ckpt/429_3_epoch=10-val_loss=0.13-val_RMSE=0.69.ckpt")

# Brute-Force Testing

## All-Together Test: 3 metrics

In [69]:
def Test_3steps(model, test_df, train_dataset, metric = 'all'):
    test_time_steps = test_df.time_id.unique()
    
    pred_0 = list()
    pred_1 = list()
    pred_2 = list()

    target_0 = list()
    target_1 = list()
    target_2 = list()

    errors = dict()
    
    for i in range(19, len(test_time_steps)):
        test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df[test_df['time_id'] <= test_time_steps[i]], predict=True, stop_randomization=True, allow_missing_timesteps=True)
        test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)
        predictions, inputs = model.predict(test_dataloader, mode='prediction', return_x=True)
        
        pred = predictions.clone().detach()
        targ = inputs['decoder_target'].clone().detach()

        pred_0.append(pred[:,0])
        pred_1.append(pred[:,1])
        pred_2.append(pred[:,2])
        target_0.append(targ[:,0])
        target_1.append(targ[:,1])
        target_2.append(targ[:,2])

    if metric == 'Pearson' or metric == 'all':
        y0 = torch.cat(pred_0).reshape(1,-1)
        y1 = torch.cat(pred_1).reshape(1,-1)
        y2 = torch.cat(pred_2).reshape(1,-1)
        
        t0 = torch.cat(target_0).reshape(1,-1)
        t1 = torch.cat(target_1).reshape(1,-1)
        t2 = torch.cat(target_2).reshape(1,-1)
        
        z0 = torch.cat((y0, t0), dim=0) # 2, B*T+
        z1 = torch.cat((y1, t1), dim=0) # 2, B*T+
        z2 = torch.cat((y2, t2), dim=0) # 2, B*T+
        
        pearson_0 = torch.corrcoef(z0)[0][1].item()
        pearson_1 = torch.corrcoef(z1)[0][1].item()
        pearson_2 = torch.corrcoef(z2)[0][1].item()
        
        errors['pearson_0'] = pearson_0
        errors['pearson_1'] = pearson_1
        errors['pearson_2'] = pearson_2
        
        print('1-step ahead Pearson correlation is')
        print(pearson_0)

        print('2-step ahead Pearson correlation is')
        print(pearson_1)

        print('3-step ahead Pearson correlation is')
        print(pearson_2)

    if metric == 'SMAPE' or metric == 'all':
        y0 = torch.cat(pred_0).reshape(-1)
        y1 = torch.cat(pred_1).reshape(-1)
        y2 = torch.cat(pred_2).reshape(-1)
        
        t0 = torch.cat(target_0).reshape(-1)
        t1 = torch.cat(target_1).reshape(-1)
        t2 = torch.cat(target_2).reshape(-1)

        smape_0 = torch.mean(2 * (y0 - t0).abs() / (y0.abs() + t0.abs() + 1e-8)).item()
        smape_1 = torch.mean(2 * (y1 - t1).abs() / (y1.abs() + t1.abs() + 1e-8)).item()
        smape_2 = torch.mean(2 * (y2 - t2).abs() / (y2.abs() + t2.abs() + 1e-8)).item()
        
        errors['smape_0'] = smape_0
        errors['smape_1'] = smape_1
        errors['smape_2'] = smape_2
        
        print('1-step ahead SMAPE is')
        print(smape_0)

        print('2-step ahead SMAPE is')
        print(smape_1)

        print('3-step ahead SMAPE is')
        print(smape_2)

    if metric == 'SMAPE' or metric == 'all':
        y0 = torch.cat(pred_0).reshape(-1)
        y1 = torch.cat(pred_1).reshape(-1)
        y2 = torch.cat(pred_2).reshape(-1)
        
        t0 = torch.cat(target_0).reshape(-1)
        t1 = torch.cat(target_1).reshape(-1)
        t2 = torch.cat(target_2).reshape(-1)

        rmse_0 = torch.sqrt(torch.mean(torch.pow(y0 - t0, 2))).item()
        rmse_1 = torch.sqrt(torch.mean(torch.pow(y1 - t1, 2))).item()
        rmse_2 = torch.sqrt(torch.mean(torch.pow(y2 - t2, 2))).item()
        
        errors['rmse_0'] = rmse_0
        errors['rmse_1'] = rmse_1
        errors['rmse_2'] = rmse_2
        
        print('1-step ahead RMSE is')
        print(rmse_0)

        print('2-step ahead RMSE is')
        print(rmse_1)

        print('3-step ahead RMSE is')
        print(rmse_1)

    return errors

In [70]:
Test_3steps(model=tft_model, test_df=test_df, train_dataset=train_dataset, metric = 'all')

1-step ahead Pearson correlation is
tensor(0.0050)
2-step ahead Pearson correlation is
tensor(0.0176)
3-step ahead Pearson correlation is
tensor(-0.0135)
1-step ahead SMAPE is
tensor(1.6089)
2-step ahead SMAPE is
tensor(1.6070)
3-step ahead SMAPE is
tensor(1.6164)
1-step ahead RMSE is
tensor(0.9130)
2-step ahead RMSE is
tensor(0.8662)
3-step ahead RMSE is
tensor(0.8662)


{'pearson_0': tensor(0.0050),
 'pearson_1': tensor(0.0176),
 'pearson_2': tensor(-0.0135),
 'smape_0': tensor(1.6089),
 'smape_1': tensor(1.6070),
 'smape_2': tensor(1.6164),
 'rmse_0': tensor(0.9130),
 'rmse_1': tensor(0.8662),
 'rmse_2': tensor(0.9254)}

## All-Together Test: Quantile Loss

In [111]:
def Test_3steps_quantiles(model, test_df, train_dataset):
    
    test_time_steps = test_df.time_id.unique()
    
    losses_0 = list()
    losses_1 = list()
    losses_2 = list()

    errors = dict()
    
    quantiles = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]

    for i in range(19, len(test_time_steps)):
        test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df[test_df['time_id'] <= test_time_steps[i]], predict=True, stop_randomization=True, allow_missing_timesteps=True)
        test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)
        predictions, inputs = model.predict(test_dataloader, mode='quantiles', return_x=True)
        
        y_pred = predictions.clone().detach() # 50 * 3 * 7
        target = inputs['decoder_target'].clone().detach() # 50 * 3

        losses = list()
        for i, q in enumerate(quantiles):
            errors = target - y_pred[..., i]
            losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(-1))
        losses = torch.cat(losses, dim=2) # 50 * 3 * 7

        losses_0.append(torch.mean(losses.clone().detach()[:,0,:]))
        losses_1.append(torch.mean(losses.clone().detach()[:,1,:]))
        losses_2.append(torch.mean(losses.clone().detach()[:,2,:]))
        
    loss_0 = torch.mean(torch.stack(losses_0)).item()
    loss_1 = torch.mean(torch.stack(losses_1)).item()
    loss_2 = torch.mean(torch.stack(losses_2)).item()

    print(loss_0)

    errors['quantile_0'] = loss_0
    errors['quantile_1'] = loss_1
    errors['quantile_2'] = loss_2

    print('1-step ahead Quantile Loss is')
    print(loss_0)

    print('2-step ahead Quantile Loss is')
    print(loss_1)

    print('3-step ahead Quantile Loss is')
    print(loss_2)

    return errors

In [ ]:
Test_3steps_quantiles(model=tft_model, test_df=test_df, train_dataset=train_dataset)

## By Investment_id Test

In [ ]:
def Test_3steps(model, test_df, train_dataset, metric = 'all'):
    test_time_steps = test_df.time_id.unique()
    
    pred_0 = list()
    pred_1 = list()
    pred_2 = list()

    target_0 = list()
    target_1 = list()
    target_2 = list()

    errors = dict()
    
    for i in range(19, len(test_time_steps)):
        test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df[test_df['time_id'] <= test_time_steps[i]], predict=True, stop_randomization=True, allow_missing_timesteps=True)
        test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)
        predictions, inputs = model.predict(test_dataloader, mode='prediction', return_x=True)
        
        pred = predictions.clone().detach()
        targ = inputs['decoder_target'].clone().detach()

        pred_0.append(pred[:,0])
        pred_1.append(pred[:,1])
        pred_2.append(pred[:,2])
        target_0.append(targ[:,0])
        target_1.append(targ[:,1])
        target_2.append(targ[:,2])

    if metric == 'Pearson' or metric == 'all':
        y0 = torch.cat(pred_0).reshape(1,-1)
        y1 = torch.cat(pred_1).reshape(1,-1)
        y2 = torch.cat(pred_2).reshape(1,-1)
        
        t0 = torch.cat(target_0).reshape(1,-1)
        t1 = torch.cat(target_1).reshape(1,-1)
        t2 = torch.cat(target_2).reshape(1,-1)
        
        z0 = torch.cat((y0, t0), dim=0) # 2, B*T+
        z1 = torch.cat((y1, t1), dim=0) # 2, B*T+
        z2 = torch.cat((y2, t2), dim=0) # 2, B*T+
        
        pearson_0 = torch.corrcoef(z0)[0][1].item()
        pearson_1 = torch.corrcoef(z1)[0][1].item()
        pearson_2 = torch.corrcoef(z2)[0][1].item()
        
        errors['pearson_0'] = pearson_0
        errors['pearson_1'] = pearson_1
        errors['pearson_2'] = pearson_2
        
        print('1-step ahead Pearson correlation is')
        print(pearson_0)

        print('2-step ahead Pearson correlation is')
        print(pearson_1)

        print('3-step ahead Pearson correlation is')
        print(pearson_2)

    if metric == 'SMAPE' or metric == 'all':
        y0 = torch.cat(pred_0).reshape(-1)
        y1 = torch.cat(pred_1).reshape(-1)
        y2 = torch.cat(pred_2).reshape(-1)
        
        t0 = torch.cat(target_0).reshape(-1)
        t1 = torch.cat(target_1).reshape(-1)
        t2 = torch.cat(target_2).reshape(-1)

        smape_0 = torch.mean(2 * (y0 - t0).abs() / (y0.abs() + t0.abs() + 1e-8)).item()
        smape_1 = torch.mean(2 * (y1 - t1).abs() / (y1.abs() + t1.abs() + 1e-8)).item()
        smape_2 = torch.mean(2 * (y2 - t2).abs() / (y2.abs() + t2.abs() + 1e-8)).item()
        
        errors['smape_0'] = smape_0
        errors['smape_1'] = smape_1
        errors['smape_2'] = smape_2
        
        print('1-step ahead SMAPE is')
        print(smape_0)

        print('2-step ahead SMAPE is')
        print(smape_1)

        print('3-step ahead SMAPE is')
        print(smape_2)

    if metric == 'SMAPE' or metric == 'all':
        y0 = torch.cat(pred_0).reshape(-1)
        y1 = torch.cat(pred_1).reshape(-1)
        y2 = torch.cat(pred_2).reshape(-1)
        
        t0 = torch.cat(target_0).reshape(-1)
        t1 = torch.cat(target_1).reshape(-1)
        t2 = torch.cat(target_2).reshape(-1)

        rmse_0 = torch.sqrt(torch.mean(torch.pow(y0 - t0, 2))).item()
        rmse_1 = torch.sqrt(torch.mean(torch.pow(y1 - t1, 2))).item()
        rmse_2 = torch.sqrt(torch.mean(torch.pow(y2 - t2, 2))).item()
        
        errors['rmse_0'] = rmse_0
        errors['rmse_1'] = rmse_1
        errors['rmse_2'] = rmse_2
        
        print('1-step ahead RMSE is')
        print(rmse_0)

        print('2-step ahead RMSE is')
        print(rmse_1)

        print('3-step ahead RMSE is')
        print(rmse_1)

    return errors

In [102]:
test_df[(test_df['time_id'] <= test_time_steps[16]) & (test_df['time_id'] >= test_time_steps[16])].investment_id.unique()

array(['34', '35', '89', '306', '322', '466', '615', '728', '861', '1062',
       '1106', '1113', '1117', '1151', '1281', '1288', '1438', '1444',
       '1544', '1555', '1568', '1607', '1766', '1997', '2120', '2422',
       '2560', '2572', '2599', '2635', '2785', '2877', '2921', '2987',
       '3012', '3029', '3060', '3082', '3160', '3251', '3255', '3331',
       '3352', '3441', '3492', '3634', '3644', '3690', '3702'],
      dtype=object)

In [ ]:
print(inputs)

In [ ]:
test_time_steps = test_df.time_id.unique()

pred_0 = list()
pred_1 = list()
pred_2 = list()

target_0 = list()
target_1 = list()
target_2 = list()

errors = dict()

for i in range(19, len(test_time_steps)):
    test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df[test_df['time_id'] <= test_time_steps[i]], predict=True, stop_randomization=True, allow_missing_timesteps=True)
    test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)
    predictions, inputs = tft_model.predict(test_dataloader, mode='prediction', return_x=True)

    pred = predictions.clone().detach()
    targ = inputs['decoder_target'].clone().detach()

    pred_0.append(pred[:,0])
    pred_1.append(pred[:,1])
    pred_2.append(pred[:,2])
    target_0.append(targ[:,0])
    target_1.append(targ[:,1])
    target_2.append(targ[:,2])


In [ ]:

if metric == 'Pearson' or metric == 'all':
    y0 = torch.cat(pred_0).reshape(1,-1)
    y1 = torch.cat(pred_1).reshape(1,-1)
    y2 = torch.cat(pred_2).reshape(1,-1)

    t0 = torch.cat(target_0).reshape(1,-1)
    t1 = torch.cat(target_1).reshape(1,-1)
    t2 = torch.cat(target_2).reshape(1,-1)

    z0 = torch.cat((y0, t0), dim=0) # 2, B*T+
    z1 = torch.cat((y1, t1), dim=0) # 2, B*T+
    z2 = torch.cat((y2, t2), dim=0) # 2, B*T+

    pearson_0 = torch.corrcoef(z0)[0][1].item()
    pearson_1 = torch.corrcoef(z1)[0][1].item()
    pearson_2 = torch.corrcoef(z2)[0][1].item()

    errors['pearson_0'] = pearson_0
    errors['pearson_1'] = pearson_1
    errors['pearson_2'] = pearson_2

    print('1-step ahead Pearson correlation is')
    print(pearson_0)

    print('2-step ahead Pearson correlation is')
    print(pearson_1)

    print('3-step ahead Pearson correlation is')
    print(pearson_2)


In [ ]:
test_df[]

In [87]:
y0s = torch.stack(pred_0, 0)

RuntimeError: stack expects each tensor to be equal size, but got [50] at entry 0 and [49] at entry 16

In [37]:
test_time_steps = test_df.time_id.unique()

In [40]:
test_time_steps

array([1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110,
       1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121,
       1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132,
       1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143,
       1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154,
       1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165,
       1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176,
       1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187,
       1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198,
       1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209,
       1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219])

In [50]:
pred_0 = list()
pred_1 = list()
pred_2 = list()

target_0 = list()
target_1 = list()
target_2 = list()

for i in range(19, len(test_time_steps)):
    test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df[test_df['time_id'] <= test_time_steps[i]], predict=True, stop_randomization=True, allow_missing_timesteps=True)
    test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)
    predictions, inputs = tft_model.predict(test_dataloader, mode='prediction', return_x=True)
    pred = predictions.clone().detach()
    targ = inputs['decoder_target'].clone().detach()
    
    pred_0.append(pred[:,0])
    pred_1.append(pred[:,1])
    pred_2.append(pred[:,2])
    target_0.append(targ[:,0])
    target_1.append(targ[:,1])
    target_2.append(targ[:,2])

In [51]:
print(pred_0[0].shape)

torch.Size([50])


In [53]:
t0 = torch.cat(target_0).reshape(1,-1)
t1 = torch.cat(target_1).reshape(1,-1)
t2 = torch.cat(target_2).reshape(1,-1)

In [71]:
test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df[test_df['time_id'] <= test_time_steps[19]], predict=True, stop_randomization=True, allow_missing_timesteps=True)

In [72]:
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)

In [73]:
predictions, inputs = tft_model.predict(test_dataloader, mode='quantiles', return_x=True)

In [74]:
print(predictions.shape)

torch.Size([50, 3, 7])


In [77]:
print(inputs['decoder_target'].shape)

torch.Size([50, 3])


In [82]:


50 100 7 mean 
 
50 100 7

50 100 7



torch.Size([50, 3, 7])


In [44]:
y1 = predictions.reshape(1, -1)
t1 = inputs['decoder_target'].reshape(1, -1)
z = torch.cat((y1, t1), dim=0) # 2, B*T+
error = torch.corrcoef(z)[0][1]

print(error)


tensor(0.0849)


In [49]:
print(predictions.clone().detach()[:,0].shape)

torch.Size([50])


In [67]:
print(y1.shape)
print(t1.shape)

torch.Size([5036])
torch.Size([5036])


In [ ]:
train_dataset.get_parameters()

In [68]:
rmse_0 = torch.pow(y0 - t0, 2)
print(rmse_0.shape)

torch.Size([5036])


# Test 2.0

In [10]:
# What's important here is that we set *predict=False*.
test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_df, predict=False, stop_randomization=True, allow_missing_timesteps=True)

NameError: name 'train_dataset' is not defined

In [ ]:
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=200, num_workers=4)

In [ ]:
for x, _ in test_dataloader:
#    tft_model.forward(x)
    print(x)
    break
#tft_model.forward(test_dataloader, mode='prediction', return_x=True)

In [ ]:
y0 = predictions[:, 0].reshape(1, -1)
y1 = predictions[:, 1].reshape(1, -1)
y2 = predictions[:, 2].reshape(1, -1)


# replace all NaN with 0
y0 = torch.nan_to_num(y0)
y1 = torch.nan_to_num(y1)
y2 = torch.nan_to_num(y2)



t0 = inputs['decoder_target'][:, 0].reshape(1, -1)
t1 = inputs['decoder_target'][:, 1].reshape(1, -1)
t2 = inputs['decoder_target'][:, 2].reshape(1, -1)

z0 = torch.cat((y0, t0), dim=0) # 2, B*T+
z1 = torch.cat((y1, t1), dim=0) # 2, B*T+
z2 = torch.cat((y2, t2), dim=0) # 2, B*T+

error_0 = torch.corrcoef(z0)[0][1]
error_1 = torch.corrcoef(z1)[0][1]
error_2 = torch.corrcoef(z2)[0][1]


print('1-step ahead prediction correlation is')
print(error_0)
      
print('2-step ahead prediction correlation is')
print(error_1)

print('3-step ahead prediction correlation is')
print(error_2)


# Test 1.0

I use data with time_id >= 1000 for testing. 
It turns out that there are 120 different time_id's for test data. 
I divde them into 6 parts: 0-19, 0-39, 0-59, 0-79, 0-99, 0-119.
Then run predictions on them.

I use encoder length = 14, decoder length = 3. 
Per my understanding, the model will tune itself on previous data and then make predictions on the last 3 time steps for each investment id. 
For example, take 0-59. The model will tune itself on 0-42 (not sure about this, it seems that it does make use of these data, but I'm not sure how), encode 43-56, and predict 57-59.  

#### Create Test DataFrames

In [ ]:
test_dfs = dict()
for i in range(6):
    test_dfs['test_df_' + str(i)] = test_df.head(math.floor(((i+1)/6) *len(test_df. index)))
    
print(test_dfs)

## Test using correlation

In [ ]:
errors = list()

for i in range(6):
    # create test_dataset from test_dfs, with parameter from train_dataset
    test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_dfs['test_df_' + str(i)], predict=True, stop_randomization=True, allow_missing_timesteps=True)
    test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)

    # make predictions
    predictions, inputs = tft_model.predict(test_dataloader, mode='prediction', return_x=True)

    # calculate correlation 
    y1 = predictions.reshape(1, -1)
    t1 = inputs['decoder_target'].reshape(1, -1)
    z = torch.cat((y1, t1), dim=0) # 2, B*T+
    error = torch.corrcoef(z)[0][1]

    errors.append(error)


In [ ]:
print(errors)